In [ ]:
from morty.extras.foldgenerator import FoldGenerator
from morty.pitchdistribution import PitchDistribution as pd
from morty.chordia import Chordia
from morty.evaluator import Evaluator
import json
import os


In [ ]:
# data
data_folder = 'data'
annotation_file = os.path.join(data_folder, 'annotations.json')
annotations = json.load(open(annotation_file, 'r'))


In [ ]:
# divide the data into stratified 5-fold
random_state = 1  # for reproducability
folds = FoldGenerator.stratified_k_fold(data_folder, annotations, n_folds=5, 
                                        random_state=random_state)


In [ ]:
# instantiate the Bozkurt classifier object
step_size = 7.5
smooth_factor = 15
distribution_type = "pcd"
distance_method = "bhat"
rank = 1
k_param = 3
chordia = Chordia(step_size=step_size, smooth_factor=smooth_factor, 
                  feature_type=distribution_type)


In [ ]:
# train, test and evaluate the method for each fold
%pdb
for fold in folds:
    chordia.train(**fold['training'])
    
    # testing
    testing_data = fold['testing']
    for t in testing_data:
        # tonic identification
        tonic = chordia.estimate_tonic(t['pitch'], t['mode'])
        
        # mode recognition
        mode = chordia.estimate_mode(t['pitch'], t['tonic'])

        # joint estimation
        joint = chordia.estimate_joint(t['pitch'])
        
        print (mode, t['mode'])
        print (tonic, t['tonic'])
        print ''
#         print joint
#         print (t['tonic'], t['mode'])
        

In [ ]:
# Initializations
ev = Evaluator()
pitch_path = "data"
mode_list = ["Hicaz", "Huseyni", "Saba"]
train_dir = "train_dir"

# Metadata of 5-fold Cross Validation Folds
annotations = json.load(open(os.path.join(pitch_path, 'annotations.json'), 'r'))
tonic_annotations = {annot['mbid']: annot['tonic'] for annot in annotations}
folds = [json.load(open(os.path.join("folds", ("fold_%d.json"%i)), 'r')) 
         for i in range(1,6)]
test_folds = [fold['test'] for fold in folds]
train_folds = [fold['train'] for fold in folds]

# Parameter Set of the Experiment
step_size = 7.5
smooth_factor = 15
distribution_type = "pcd"
distance_method = "bhat"
k_param = 3
chordia = Chordia(step_size=step_size, smooth_factor=smooth_factor, chunk_size=0)

# Identifier String of this Parameter Set
exp_str = ("sf_%2.1f__ss_%2.1f__%s__dist_%s__k_%d" % 
           (smooth_factor, step_size, distribution_type, 
            distance_method, k_param)).replace('.', '')


In [ ]:
for fold_idx in range(5):
    train_fold = train_folds[fold_idx]
    test_fold = test_folds[fold_idx]
    
    # Training
    for mode in mode_list:            
        train_mbids = [track['mbid'] for track in train_fold if(track['makam'] == mode)]
        train_tonics = [tonic_annotations[mbid] for mbid in train_mbids]
        train_paths = [os.path.join(pitch_path, mode, (mbid+".pitch")) 
                       for mbid in train_mbids]
        
        chordia.train(mode, train_paths, train_tonics, feature=distribution_type, 
                      save_dir=train_dir)
    
    # Testing    
    for test_pitch in test_fold:
        test_path = os.path.join(pitch_path, test_pitch["makam"], 
                                 (test_pitch["mbid"]+".pitch"))
        
        # Mode Recognition
        mode_est = chordia.mode_estimate(
            test_path, test_pitch["tonic"], mode_names=mode_list,
            mode_dir=train_dir, distance_method=distance_method,
            feature=distribution_type, k_param=k_param)
        
        # Tonic Identification
        tonic_est = chordia.tonic_estimate(
            test_path, mode, mode_dir=train_dir, 
            distance_method=distance_method, feature=distribution_type, 
            k_param=k_param)
        
        # Joint Estimation
        joint_tonic_est, joint_mode_est = chordia.joint_estimate(
            test_path, mode_list, mode_dir=train_dir, 
            distance_method=distance_method, feature=distribution_type, 
            k_param=k_param)
        
        # Evaluations
        mode_eval = (mode_est == test_pitch["makam"])
        tonic_eval = ev.tonic_evaluate(test_pitch["mbid"], tonic_est, 
                                       test_pitch["tonic"])["tonic_eval"]
        joint_eval = ((mode_est == test_pitch["makam"]), 
                      ev.tonic_evaluate(test_pitch["mbid"], joint_tonic_est,
                                        test_pitch["tonic"])["tonic_eval"])
        
        print("mbid:%s" % test_pitch["mbid"])
        print("\nMode Recognition:\tEstimated:%s\tActual:%s\tEvaluation:%s" % 
              (mode_est, test_pitch["makam"], str(mode_eval)))
        print("Tonic Identification:\tEstimated:%4.2f\tActual:%4.2f\tEvaluation:%s\n" % 
              (tonic_est, test_pitch["tonic"], str(tonic_eval)))
        print(("Joint Estimation:\nMode:\tEstimated:%s\tActual:%s\tEvaluation:%s\n"
               "Tonic:\tEstimated:%4.2f\tActual:%4.2f\tEvaluation:%s\n") % 
              (joint_mode_est, test_pitch["makam"], str(joint_eval[0]),
               joint_tonic_est, test_pitch["tonic"], str(joint_eval[1])))
        print("----------------------------------------------------------------------")
